# NLP From Scratch

## Translation with a Sequence to Sequence Network and Attention
===============================================================================

In this project we will be teaching a neural network to translate from
Portuguese to English.

``` {.sh}
[KEY: > input, = target]

> Ele gosta de jogar futebol .
= He likes playing football .

> Eu sei contar até cem .
= I know how to count to 100 .

> Não fui capaz de encontrar o caminho .
= I wasn't able to find my way out .
```

This is made possible by the simple but powerful idea of the [sequence
to sequence network](https://arxiv.org/abs/1409.3215), in which two
recurrent neural networks work together to transform one sequence to
another. An encoder network condenses an input sequence into a vector,
and a decoder network unfolds that vector into a new sequence.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/seq2seq.png)

To improve upon this model we\'ll use an [attention
mechanism](https://arxiv.org/abs/1409.0473), which lets the decoder
learn to focus over a specific range of the input sequence.


Loading data files
==================

The data for this project is a set of many thousands of English to
Portuguese translation pairs.

Individual files with language pairs are available here: <https://www.manythings.org/anki/>

The English to Portuguese pairs are available in this repository in the `por-eng` directory with the filename `por.txt`.



We will be representing each word in a language as a one-hot
vector. Compared to the dozens of characters that might exist in a
language, there are many many more words, so the encoding vector is much
larger. We will however cheat a bit and trim the data to only use a few
thousand words per language.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/word-encoding.png)


We\'ll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called `Lang` which has word → index (`word2index`) and index → word
(`index2word`) dictionaries, as well as a count of each word
`word2count` which will be used to replace rare words later.


## Exercise 1: `Lang` class

### 1. Implement the `Lang` class with the following methods:
   - `__init__(self, name)`
   - `add_sentence(self, sentence)`
   - `addWord(self, word)`

The `Lang` class should have the following attributes:
    - `name`: the name of the language
    - `word2index`: a dictionary mapping words to indexes, default = `{}`
    - `word2count`: a dictionary mapping words to their count, default = `{}`
    - `index2word`: a dictionary mapping indexes to words, default = `{0: "SOS", 1: "EOS"}`
    - `n_words`: the number of words in the language, default = `2`
    
The `addSentence` method should split the sentence into words and call the `addWord` method for each word.

The `add_word` method should add the word to the `word2index`, `word2count`, and `index2word` dictionaries if it is not already in the `word2index` dictionary. If the word is already in the `word2index` dictionary, increment the count of the word in the `word2count` dictionary.
    

In [1]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        # ...

    def add_sentence(self, sentence):
        # ...

    def add_word(self, word):
        # ...

## Exercice 2: Load Data and Text Processing

### 1. Normalize the text 

Implement the following function:

- `normalize_string(string)`: Lowercase, trim (strip), separate ".", "!", "?", and words with spaces. Replace all non-letter characters with spaces (except for ".", "!", "?"). Example: "I am a student." -> "i am a student ."




In [2]:
import re

def normalize_string(string):
    # ...
    return string

### 2. Load the data

Implement the following function: 

- `load_data()`: Read the file and split into lines, split lines into pairs (eng-pt), and normalize each pair. Return the pairs.


In [3]:
def load_data():
    # ...
    return pairs
    
load_data()[:5]

[('go .', 'vai .'),
 ('go .', 'vá .'),
 ('hi .', 'oi .'),
 ('run !', 'corre !'),
 ('run !', 'corra !')]

### 3. Filter the data

Since there are a *lot* of example sentences and we want to train
something quickly, we\'ll trim the data set to only relatively short and
simple sentences. Here the maximum length is 10 words (that includes
ending punctuation).

Implement the following function:

- `filter_pair(p)`: Return `True` if the pair is shorter than the maximum length, i.e, if the length of the first sentence is less than 10 and the length of the second sentence is less than 10.

- `filter_pairs(pairs)`: Return a list of pairs that satisfy the condition of the `filter_pair` function.




In [4]:
MAX_LENGTH = 10


def filter_pair(p):
    return # ...


def filter_pairs(pairs):
    return # ...

### 4. All the data processing steps

Implement the following function:

- `prepare_data(lang1, lang2)`: Read text file and split into lines, split lines into pairs, normalize text, filter by length, and make word lists from sentences in pairs. Return the input language (`Lang` object), output language (`Lang` object), and pairs.

The full process for preparing the data is:

-   Read text file and split into lines, split lines into pairs
-   Normalize text, filter by length
-   Make word lists from sentences in pairs


In [5]:
import random


def prepare_data(lang1, lang2):
    # ...
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepare_data('pt', 'eng')
print(random.choice(pairs))

Read 193633 sentence pairs
Trimmed to 154611 sentence pairs
Counting words...
Counted words:
pt 19422
eng 11442
('how did you get back so soon ?', 'como foi que você voltou tão rápido ?')


# The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A [Sequence to Sequence network](https://arxiv.org/abs/1409.3215), or
seq2seq network, or [Encoder Decoder
network](https://arxiv.org/pdf/1406.1078v3.pdf), is a model consisting
of two RNNs called the encoder and decoder. The encoder reads an input
sequence and outputs a single vector, and the decoder reads that vector
to produce an output sequence.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/seq2seq.png)

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

Consider the sentence `Eu não sou o gato preto` →
`I am not the black cat`. Most of the words in the input sentence have a
direct translation in the output sentence, but are in slightly different
orders, e.g. `gato preto` and `black cat`. Additionally, sometimes the length
of the input sequence is different from the output sequence.

It would be difficult to produce a correct translation directly from the sequence
of input words.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the \"meaning\" of the input sequence into a single
vector --- a single point in some N dimensional space of sentences.


The Encoder
===========

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/encoder-network.png)


## Exercice 3: EncoderRNN class

### 1. Implement the `EncoderRNN` class with the following methods:
   - `__init__(self, input_size, hidden_size, dropout_p=0.1)`: Initialize the encoder with the input size, hidden size, and dropout probability. The encoder should have an embedding layer, a GRU layer, and a dropout layer.
   - `forward(self, input)`: Forward pass of the encoder. The input is passed through an embedding layer, followed by a GRU layer. The output and hidden state of the GRU layer are returned.

In [6]:
import torch.nn as nn

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        # ...

    def forward(self, input):
        # ...
        return output, hidden

The Decoder
===========

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.


Simple Decoder
==============

In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string `<SOS>`
token, and the first hidden state is the context vector (the encoder\'s
last hidden state).

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/decoder-network.png)


## Exercice 4: DecoderRNN class

### 1. Implement the `DecoderRNN` class with the following methods:
   - `__init__(self, hidden_size, output_size)`: Initialize the decoder with the hidden size and output size. The decoder should have an embedding layer, a GRU layer, and a linear layer.
   - `forward(self, encoder_outputs, encoder_hidden, target_tensor=None)`: Forward pass of the decoder. The decoder takes the encoder outputs, encoder hidden state, and target tensor as input. The target tensor is used for teacher forcing. The decoder outputs are returned.
   - `forward_step(self, input, hidden)`: Forward pass of the decoder for a single step. The input is passed through an embedding layer, followed by a GRU layer, and a linear layer. The output and hidden state of the GRU layer are returned.

In [7]:
import torch
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        # ...

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        # ...
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        # ...
        return output, hidden

cuda


Attention Decoder
=================

If only the context vector is passed between the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to \"focus\" on a different part of
the encoder\'s outputs for every step of the decoder\'s own outputs.
First we calculate a set of *attention weights*. These will be
multiplied by the encoder output vectors to create a weighted
combination. The result (called `attn_applied` in the code) should
contain information about that specific part of the input sequence, and
thus help the decoder choose the right output words.

![](img/1152PYf.png)

Calculating the attention weights is done with another feed-forward
layer `attn`, using the decoder\'s input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.

![](https://pytorch.org/tutorials/_static/img/seq-seq-images/attention-decoder-network.png)

Bahdanau attention, also known as additive attention, is a commonly used
attention mechanism in sequence-to-sequence models, particularly in
neural machine translation tasks. It was introduced by Bahdanau et al.
in their paper titled [Neural Machine Translation by Jointly Learning to
Align and Translate](https://arxiv.org/pdf/1409.0473.pdf). This
attention mechanism employs a learned alignment model to compute
attention scores between the encoder and decoder hidden states. It
utilizes a feed-forward neural network to calculate alignment scores.

However, there are alternative attention mechanisms available, such as
Luong attention, which computes attention scores by taking the dot
product between the decoder hidden state and the encoder hidden states.
It does not involve the non-linear transformation used in Bahdanau
attention.

In this tutorial, we will be using Bahdanau attention. However, it would
be a valuable exercise to explore modifying the attention mechanism to
use Luong attention.


## Exercice 5: BahdanauAttention and AttnDecoderRNN classes

### 1. Implement the `BahdanauAttention` class with the following methods:
   - `__init__(self, hidden_size)`: Initialize the attention mechanism with the hidden size. The attention mechanism should have three linear layers.
   - `forward(self, query, keys)`: Forward pass of the attention mechanism. The query and keys are passed through the linear layers and the attention weights are calculated. The context vector and attention weights are returned.
   
### 2. Implement the `AttnDecoderRNN` class with the following methods:
   - `__init__(self, hidden_size, output_size, dropout_p=0.1)`: Initialize the decoder with the hidden size, output size, and dropout probability. The decoder should have an embedding layer, an attention mechanism, a GRU layer, and a linear layer.
  - `forward(self, encoder_outputs, encoder_hidden, target_tensor=None)`: Forward pass of the decoder. The decoder takes the encoder outputs, encoder hidden state, and target tensor as input. The target tensor is used for teacher forcing. The decoder outputs are returned.
- `forward_step(self, input, hidden)`: Forward pass of the decoder for a single step. The input is passed through an embedding layer, followed by the attention mechanism, a GRU layer, and a linear layer. The output and hidden state of the GRU layer are returned.s

In [8]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        # ...

    def forward(self, query, keys):
        # ...
        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        # ...

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        # ...
        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        # ...
        return output, hidden, attn_weights

Training
========

Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.


## Exercice 6: Indexes and Tensors

### 1. Implement the following functions:
- `indexes_from_sentence(lang, sentence)`: Return a list of indexes from the sentence.
- `tensorFromSentence(lang, sentence)`: Return a tensor from the sentence.
- `tensors_from_pair(pair)`: Return a pair of tensors from the pair.

In [9]:
from torch.utils.data import TensorDataset, RandomSampler, DataLoader


def indexes_from_sentence(lang, sentence):
    return # ...

def tensorFromSentence(lang, sentence):
    # ...
    return indexes

def tensors_from_pair(pair):
    # ...
    return (input_tensor, target_tensor)

def get_dataloader(batch_size):
    # ...
    return input_lang, output_lang, train_dataloader

Training the Model
==================

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the `<SOS>` token as its first input, and the last hidden state of the
encoder as its first hidden state.

\"Teacher forcing\" is the concept of using the real target outputs as
each next input, instead of using the decoder\'s guess as the next
input. Using teacher forcing causes it to converge faster but [when the
trained network is exploited, it may exhibit
instability](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.378.4095&rep=rep1&type=pdf).

You can observe outputs of teacher-forced networks that read with
coherent grammar but wander far from the correct translation
-intuitively it has learned to represent the output grammar and can
\"pick up\" the meaning once the teacher tells it the first few words,
but it has not properly learned how to create the sentence from the
translation in the first place.

Because of the freedom PyTorch\'s autograd gives us, we can randomly
choose to use teacher forcing or not with a simple if statement. Turn
`teacher_forcing_ratio` up to use more of it.


## Exercice 7: Training

### 1. Implement the following functions:
- `train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)`: Train the model for one epoch.


In [10]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    # ...
    return loss

## Exercice 8: Helper functions

### 1. Implement the following functions:
- `as_minutes(s)`: Convert seconds to minutes.
- `time_since(since, percent)`: Calculate the time since the start of training.


In [11]:
import time
import math

def as_minutes(s):
    # ...
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    # ...
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

## Exercice 9: Training

### 1. Implement the following function:
- `train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=100, plot_every=100)`: Train the model for a number of epochs.

The whole training process looks like this:

-   Start a timer
-   Initialize optimizers and criterion
-   Create set of training pairs
-   Start empty losses array for plotting

Then we call `train` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.


In [12]:
from torch import optim


def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=100, plot_every=100):
    # ...

## Exercice 10: Plotting

### 1. Implement the following function:
- `show_plot(points)`: Plot the points.

================

Plotting is done with matplotlib, using the array of loss values
`plot_losses` saved while training.


In [13]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def show_plot(points):
    # ...

## Exercice 11: Evaluation

### 1. Implement the following functions:

- `evaluate(encoder, decoder, sentence, input_lang, output_lang)`: Evaluate the model on a sentence.

==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder\'s predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder\'s
attention outputs for display later.


In [14]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    # ...
    return decoded_words, decoder_attn

## Exercice 12: Random Evaluation

### 1. Implement the following function:
- `evaluate_randomly(encoder, decoder, n=10)`: Evaluate the model on random sentences.

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:


In [15]:
def evaluate_randomly(encoder, decoder, n=10):
    # ...

Training and Evaluating
=======================

With all these helper functions in place (it looks like extra work, but
it makes it easier to run multiple experiments) we can actually
initialize a network and start training.

Remember that the input sentences were heavily filtered. For this small
dataset we can use relatively small networks of 256 hidden nodes and a
single GRU layer. After about 40 minutes on a MacBook CPU we\'ll get
some reasonable results.



## Exercice 13: Training

### 1. Train the model for 80 epochs.

In [16]:
hidden_size = 128
batch_size = 32

# ...

Read 193633 sentence pairs
Trimmed to 154611 sentence pairs
Counting words...
Counted words:
pt 19422
eng 11442
5m 56s (- 89m 1s) (5 6%) 1.0004
11m 35s (- 81m 8s) (10 12%) 0.4598
17m 4s (- 73m 59s) (15 18%) 0.3534
22m 31s (- 67m 35s) (20 25%) 0.3007
27m 56s (- 61m 28s) (25 31%) 0.2685
33m 19s (- 55m 32s) (30 37%) 0.2465
38m 43s (- 49m 46s) (35 43%) 0.2298
44m 4s (- 44m 4s) (40 50%) 0.2174
49m 28s (- 38m 29s) (45 56%) 0.2077


KeyboardInterrupt: 

###  2. Evaluate the model

Set dropout layers to `eval` mode


In [ ]:
encoder.eval()
decoder.eval()
evaluate_randomly(encoder, decoder)

Visualizing Attention
=====================

A useful property of the attention mechanism is its highly interpretable
outputs. Because it is used to weight specific encoder outputs of the
input sequence, we can imagine looking where the network is focused most
at each time step.

You could simply run `plt.matshow(attentions)` to see attention output
displayed as a matrix. For a better viewing experience we will do the
extra work of adding axes and labels:


In [ ]:
def show_attention(input_sentence, output_words, attentions):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.cpu().numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluate_and_show_attention(input_sentence):
    output_words, attentions = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    show_attention(input_sentence, output_words, attentions[0, :len(output_words), :])


evaluate_and_show_attention('il n est pas aussi grand que son pere')

evaluate_and_show_attention('je suis trop fatigue pour conduire')

evaluate_and_show_attention('je suis desole si c est une question idiote')

evaluate_and_show_attention('je suis reellement fiere de vous')